In [1]:
%matplotlib qt

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from prettyplots import pretty_axis
import pickle
from astropy.table import Table

# Plot stellar SEDs

In [10]:
alf_cen_infile = {
    'peak':'../atm/stellar_flux/alf_cen_peak.txt',
    'min':'../atm/stellar_flux/alf_cen_min.txt',
}

solar_infile = {
    'gueymard':'../atm/stellar_flux/gueymard_solar.txt',
    'woods':'../atm/stellar_flux/woods_solar.txt',
}


alf_cen_data = {
    'peak':Table.read(alf_cen_infile['peak'],format='ascii'),
    'min':Table.read(alf_cen_infile['min'],format='ascii'),
}

solar_data = {
    'gueymard':Table.read(solar_infile['gueymard'],format='ascii',names=['wavelength','flux']),
    'woods':Table.read(solar_infile['woods'],format='ascii'),
}


In [14]:
fig,ax = plt.subplots(1,1,layout='constrained')
pretty_axis(ax)

ax.loglog(solar_data['gueymard']['wavelength'],solar_data['gueymard']['flux'],label='Gueymard solar')
ax.loglog(solar_data['woods']['wavelength'],solar_data['woods']['flux'],label='Woods solar')
ax.loglog(alf_cen_data['peak']['wavelength'],alf_cen_data['peak']['flux'],label=r'$\alpha$ Cen peak')
ax.loglog(alf_cen_data['min']['wavelength'],alf_cen_data['min']['flux'],label=r'$\alpha$ Cen min')
ax.legend(fontsize=14)
ax.set_xlabel(r'Wavelength [nm]',fontsize=18)
ax.set_ylabel(r'Flux [erg s$^{-1}$ cm$^{-2}$ nm$^{-1}$]',fontsize=18)

Text(0, 0.5, 'Flux [erg s$^{-1}$ cm$^{-2}$ nm$^{-1}$]')

# Plot atmosphere retrievals

In [8]:
min_file = '../output/alf_cen_min.vul'
peak_file = '../output/alf_cen_peak.vul'
earth_file = '../output/Earth.vul'

In [13]:
specs_to_plot = ['H','He','H2','H2O','CO','CO2','CH4','N2','NH3','O','O2']

fig,ax = plt.subplots(1,2,layout='constrained',figsize=(10,8),sharey=True)
pretty_axis(ax)

for infile,linestyle,label in zip([peak_file,earth_file],['-','--'],['peak','Earth']):
    with open(infile,'rb') as handle:
        data = pickle.load(handle)
        pressure = data['atm']['pco']/1.e6
        
    mixing_ratios = {}
    
    for species in specs_to_plot:
        ind = data['variable']['species'].index(species)
        mixing_ratios[species] = data['variable']['ymix'][:,ind]

    i = 0
    for sp,mr in mixing_ratios.items():
        for char in sp:
            if char.isdigit():
                # Format species for label
                sp = sp.replace(char,"$_%s$"%(char))
        if 'peak' in infile:
            ax[0].plot(mr,pressure,label=sp,ls=linestyle,c='C%s'%(i))
        else:
            ax[0].plot(mr,pressure,ls=linestyle,c='C%s'%(i))
        i += 1

ax[0].set_yscale('log')
ax[0].set_xscale('log')
ax[0].invert_yaxis() 
ax[0].minorticks_on()
ax[0].set_ylim((data['atm']['pco'][0]/1e6,data['atm']['pco'][-1]/1e6))
ax[0].set_ylabel("Pressure [bar]",fontsize=20)
ax[0].set_xlabel("Mixing ratio",fontsize=20)
ax[0].legend(fontsize=14)

ax[1].plot(data['atm']['Tco'],pressure,label=r'$\alpha$ Cen A b')
#ax[1].plot(CH4_CO['col1'],CH4_CO['col2'],ls='--',c='r',label='CH4 / CO')
#ax[1].plot(NH3_N2['col1'],NH3_N2['col2'],ls='--',c='g',label='NH3 / N2')
ax[1].set_yscale('log')
#ax[1].invert_yaxis()
ax[1].legend(fontsize=14)
ax[1].set_xlabel('T [K]',fontsize=18)
#ax[1].set_ylabel('Pressure [bar]',fontsize=18)
ax[1].set_ylim([1,5e-8])
ax[1].set_xlim([150,350])

(150.0, 350.0)